In [1]:
import boto3


In [ ]:
import os
import boto3
from PIL import Image

# -------- CONFIG --------
BUCKET = "fuji-images"        # your bucket
REGION = "us-east-2"          # your region
LOCAL_DIR = "/Users/ashfi/Documents/FUJI Edits"          # local folder containing originals
SIZES = [960, 1600]           # target widths in px
QUALITY = 80                  # for WebP/JPEG; AVIF quality ~30-50 recommended
# ------------------------

s3 = boto3.client("s3", region_name=REGION)

def save_and_upload_variant(img, base_name, size, fmt, ext, content_type, quality):
    """Resize and upload one variant to S3"""
    img_copy = img.copy()
    img_copy.thumbnail((size, size), Image.LANCZOS)

    out_name = f"{os.path.splitext(base_name)[0]}_{size}.{ext}"
    tmp_path = f"/tmp/{out_name}"

    save_kwargs = {}
    if fmt == "JPEG":
        save_kwargs = {"quality": quality, "progressive": True}
    elif fmt == "WEBP":
        save_kwargs = {"quality": quality, "method": 6}
    elif fmt == "AVIF":
        save_kwargs = {"quality": 40}  # AVIF quality scale different

    img_copy.save(tmp_path, fmt, **save_kwargs)

    with open(tmp_path, "rb") as f:
        s3.put_object(
            Bucket=BUCKET,
            Key=out_name,
            Body=f,
            ContentType=content_type,
            CacheControl="public, max-age=31536000, immutable"
        )
    print(f"✅ Uploaded {out_name} ({fmt}, {size}px)")

def process_image(path):
    base_name = os.path.basename(path)
    try:
        with Image.open(path) as img:
            for size in SIZES:
                save_and_upload_variant(img, base_name, size, "JPEG", "jpg", "image/jpeg", QUALITY)
                save_and_upload_variant(img, base_name, size, "WEBP", "webp", "image/webp", QUALITY)
                save_and_upload_variant(img, base_name, size, "AVIF", "avif", "image/avif", QUALITY)
    except Exception as e:
        print(f"⚠️ Skipped {base_name}: {e}")

def main():
    for file in os.listdir(LOCAL_DIR):
        if file.lower().endswith((".jpg", ".jpeg", ".png")):
            process_image(os.path.join(LOCAL_DIR, file))

if __name__ == "__main__":
    main()


In [3]:

bucket_name = 'fuji-images'
region = 'us-east-2'  # e.g., 'us-east-1'

s3 = boto3.client('s3')

# List all files in the bucket
objects = s3.list_objects_v2(Bucket=bucket_name)
urls = []
# Generate public URLs (assuming your bucket is public or files are public)
if 'Contents' in objects:
    for obj in objects['Contents']:
        key = obj['Key']
        url = f"https://{bucket_name}.s3.{region}.amazonaws.com/{key}"
        urls.append(url)
else:
    print("No files found.")


In [4]:
urls

['https://fuji-images.s3.us-east-2.amazonaws.com/20220115_0055.jpg',
 'https://fuji-images.s3.us-east-2.amazonaws.com/20220115_0058.jpg',
 'https://fuji-images.s3.us-east-2.amazonaws.com/20220115_0059.jpg',
 'https://fuji-images.s3.us-east-2.amazonaws.com/20220115_0060.jpg',
 'https://fuji-images.s3.us-east-2.amazonaws.com/20220115_0062.jpg',
 'https://fuji-images.s3.us-east-2.amazonaws.com/20220115_0064.jpg',
 'https://fuji-images.s3.us-east-2.amazonaws.com/20220115_0066.jpg',
 'https://fuji-images.s3.us-east-2.amazonaws.com/20220115_0068.jpg',
 'https://fuji-images.s3.us-east-2.amazonaws.com/20220115_0069.jpg',
 'https://fuji-images.s3.us-east-2.amazonaws.com/20220115_0072.jpg',
 'https://fuji-images.s3.us-east-2.amazonaws.com/DSCF0003.jpg',
 'https://fuji-images.s3.us-east-2.amazonaws.com/DSCF0015.jpg',
 'https://fuji-images.s3.us-east-2.amazonaws.com/DSCF0031.jpg',
 'https://fuji-images.s3.us-east-2.amazonaws.com/DSCF0033.jpg',
 'https://fuji-images.s3.us-east-2.amazonaws.com/DSCF0

In [5]:
import exifread
import boto3
import pandas as pd
import os
import tempfile

s3 = boto3.client('s3')
bucket = 'fuji-images'
region = 'us-east-2'

# List S3 objects (images)
objects = s3.list_objects_v2(Bucket=bucket)
records = []

for obj in objects.get('Contents', []):
    key = obj['Key']
    if not key.lower().endswith(('.jpg', '.jpeg')):
        continue

    # Download image
    with tempfile.NamedTemporaryFile(delete=False, suffix='.jpg') as tmp:
        s3.download_fileobj(bucket, key, tmp)
        tmp_path = tmp.name

    # Extract EXIF
    with open(tmp_path, 'rb') as f:
        tags = exifread.process_file(f)
        metadata = {str(k): str(v) for k, v in tags.items()}
        metadata['filename'] = key
        records.append(metadata)

    os.remove(tmp_path)

# Save as CSV and upload to S3
df = pd.DataFrame(records)
csv_path = '/tmp/exif_metadata.csv'
df.to_csv(csv_path, index=False)

s3.upload_file(csv_path, bucket, 'metadata/exif_metadata.csv')


ModuleNotFoundError: No module named 'exifread'

In [ ]:
import os
import json
import exifread
import boto3
import pandas as pd

# Define your image directory and S3 settings
image_dir = "static/images"
bucket_name = "fuji-images"
metadata_filename = "exif_metadata.json"
region = "us-east-2"

# Initialize boto3 client
s3 = boto3.client('s3', region_name=region)

# Collect EXIF metadata for all images
image_extensions = ('.jpg', '.jpeg', '.png')
metadata_list = []

for filename in os.listdir(image_dir):
    if filename.lower().endswith(image_extensions):
        filepath = os.path.join(image_dir, filename)
        try:
            with open(filepath, 'rb') as img_file:
                tags = exifread.process_file(img_file, details=False)
                metadata = {str(tag): str(tags[tag]) for tag in tags}
                metadata['filename'] = filename
                metadata_list.append(metadata)
        except Exception as e:
            metadata_list.append({'filename': filename, 'error': str(e)})

# Save metadata to JSON
metadata_path = os.path.join(image_dir, metadata_filename)
with open(metadata_path, 'w') as f:
    json.dump(metadata_list, f, indent=2)

# Upload images and metadata to S3
for filename in os.listdir(image_dir):
    if filename.lower().endswith(image_extensions) or filename == metadata_filename:
        filepath = os.path.join(image_dir, filename)
        s3.upload_file(filepath, bucket_name, filename)

metadata_path  # Return path to the generated metadata file


'static/images/exif_metadata.json'

In [ ]:
for col in df.columns:
    print(f"{col}: {df[col].dtype}")


Image ImageDescription: object
Image Make: object
Image Model: object
Image Orientation: object
Image XResolution: object
Image YResolution: object
Image ResolutionUnit: object
Image Software: object
Image DateTime: object
Image WhitePoint: object
Image PrimaryChromaticities: object
Image YCbCrCoefficients: object
Image YCbCrPositioning: object
Image Rating: object
Image Tag 0x4749: object
Image Copyright: object
Image ExifOffset: object
Image DateTimeOriginal: object
Image PrintIM: object
EXIF ExposureTime: object
EXIF FNumber: object
EXIF ExposureProgram: object
EXIF ISOSpeedRatings: object
EXIF SensitivityType: object
EXIF ExifVersion: object
EXIF DateTimeOriginal: object
EXIF DateTimeDigitized: object
EXIF ComponentsConfiguration: object
EXIF CompressedBitsPerPixel: object
EXIF ShutterSpeedValue: object
EXIF ApertureValue: object
EXIF BrightnessValue: object
EXIF ExposureBiasValue: object
EXIF MaxApertureValue: object
EXIF MeteringMode: object
EXIF LightSource: object
EXIF Flash: o

In [ ]:
import os
import json
import exifread
import boto3

# --- CONFIGURATION ---
LOCAL_IMAGE_DIR = "static/images"
IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png')
S3_BUCKET = "fuji-images"
S3_REGION = "us-east-2"
S3_METADATA_PREFIX = "metadata/"  # folder in S3 to store metadata files

# --- SETUP S3 CLIENT ---
s3 = boto3.client("s3", region_name=S3_REGION)

def extract_exif_data(image_path):
    try:
        with open(image_path, 'rb') as f:
            tags = exifread.process_file(f, details=False)
            return {str(tag): str(tags[tag]) for tag in tags}
    except Exception as e:
        return {"Error": f"Could not extract EXIF: {str(e)}"}

def upload_metadata_to_s3(metadata, s3_key):
    try:
        s3.put_object(
            Bucket=S3_BUCKET,
            Key=s3_key,
            Body=json.dumps(metadata),
            ContentType='application/json'
        )
        print(f"✅ Uploaded: {s3_key}")
    except Exception as e:
        print(f"❌ Failed to upload {s3_key}: {e}")

def process_images():
    for filename in os.listdir(LOCAL_IMAGE_DIR):
        if filename.lower().endswith(IMAGE_EXTENSIONS):
            local_path = os.path.join(LOCAL_IMAGE_DIR, filename)
            exif_data = extract_exif_data(local_path)

            json_filename = os.path.splitext(filename)[0] + ".json"
            s3_key = os.path.join(S3_METADATA_PREFIX, json_filename)

            upload_metadata_to_s3(exif_data, s3_key)

if __name__ == "__main__":
    print("📸 Starting EXIF extraction and upload...")
    process_images()
    print("✅ Done.")


📸 Starting EXIF extraction and upload...
✅ Uploaded: metadata/_DSF1313.json
✅ Uploaded: metadata/_DSF0019.json
✅ Uploaded: metadata/DSCF3664.json
✅ Uploaded: metadata/_DSF4519.json
✅ Uploaded: metadata/DSCF2552.json
✅ Uploaded: metadata/_DSF5149.json
✅ Uploaded: metadata/_DSF0025.json
✅ Uploaded: metadata/_DSF2154.json
✅ Uploaded: metadata/_DSF0970.json
✅ Uploaded: metadata/_DSF9940.json
✅ Uploaded: metadata/DSCF4686.json
✅ Uploaded: metadata/DSCF3843.json
✅ Uploaded: metadata/_DSF1111.json
✅ Uploaded: metadata/_DSF0569.json
✅ Uploaded: metadata/DSCF5771.json
✅ Uploaded: metadata/_DSF1105.json
✅ Uploaded: metadata/DSCF3466.json
✅ Uploaded: metadata/_DSF2424.json
✅ Uploaded: metadata/DSCF2988.json
✅ Uploaded: metadata/DSCF4447.json
✅ Uploaded: metadata/_DSF1893.json
✅ Uploaded: metadata/DSCF5981.json
✅ Uploaded: metadata/DSCF5759.json
✅ Uploaded: metadata/_DSF5439.json
✅ Uploaded: metadata/_DSF1887.json
✅ Uploaded: metadata/DSCF2977.json
✅ Uploaded: metadata/DSCF0812.json
✅ Uploaded: me